In [ ]:
#pip install tensorflow

# Librerias a implementar

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import datetime
import io

from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import my_functions

# Datos usados

In [6]:
data_siniestros = pd.read_csv('Data/siniestros.csv')
data_contactos = pd.read_csv('Data/contactos.csv')
data_cobertura_2022 = pd.read_csv('Data/Data_cobertura/data_test', delimiter=',')
data_certificados = pd.read_csv('Data/certificados.csv')
data_tenencias = pd.read_csv('Data/tenencia.csv')
data_review = pd.read_csv('Data/miarchivo.csv', delimiter=',')

# Funciones de transformacion

In [11]:
data_contactos_new = my_functions.Limpieza_contactos(data_contactos)
data_siniestros_new = my_functions.Limpieza_siniestros(data_siniestros)
data_cobertura_2022_new = my_functions.Limpieza_datos_coberturas(data_cobertura_2022)
data_certificados_new = my_functions.Limpieza_certificados(data_certificados)
data_tenencias_new = my_functions.Limpieza_tenencias(data_tenencias)

c:\Users\102140\OneDrive - Grundfos\Personal\DATA SCIENCE\Proyecto Galicia\my_functions.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_wapp['DESC_GEST']= valores_de_reemplazo
c:\Users\102140\OneDrive - Grundfos\Personal\DATA SCIENCE\Proyecto Galicia\my_functions.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  data['DESCRIPCION_COBERTURA'] = data['DESCRIPCION_COBERTURA'].str.replace(r'\xa0', ' ')


In [13]:
columna_a = data_tenencias_new.pop('KEY_CLIENT')  # Elimina 'ColumnaA' de su posición original
data_tenencias_new.insert(data_tenencias_new.columns.get_loc('VIDA_OBLIGATORIO_CORE_2') + 1, 'KEY_CLIENT', columna_a)
data_y = data_tenencias_new.iloc[:,:43]

# Valores Coincidentes

In [16]:
frecuencia_columna2a = data_review['KEY_CLIENT'].value_counts()
frecuencia_columna2 = data_contactos_new['KEY_CLIENT'].value_counts()
frecuencia_columna1a = data_siniestros_new['KEY_CLIENT'].value_counts()
frecuencia_columna1b = data_cobertura_2022_new['KEY_CLIENT'].value_counts()
frecuencia_columna1e = data_certificados_new['KEY_CLIENT'].value_counts()
frecuencia_columna1f = data_y['KEY_CLIENT'].value_counts()

valores_coincidentes = frecuencia_columna2a.index.intersection(frecuencia_columna2.index)
valores_coincidentes1 = frecuencia_columna2a.index.intersection(frecuencia_columna1a.index)
valores_coincidentes2 = frecuencia_columna2a.index.intersection(frecuencia_columna1b.index)
valores_coincidentes3 = frecuencia_columna2a.index.intersection(frecuencia_columna1e.index)
valores_coincidentes4 = frecuencia_columna1b.index.intersection(frecuencia_columna1f.index)

cantidad_coincidentes = len(valores_coincidentes)
cantidad_coincidentes1 = len(valores_coincidentes1)
cantidad_coincidentes2 = len(valores_coincidentes2)
cantidad_coincidentes3 = len(valores_coincidentes3)
cantidad_coincidentes4 = len(valores_coincidentes4)

print("Cantidad de valores coincidentes Contactos:", cantidad_coincidentes)
print("Cantidad de valores coincidentes Siniestros:", cantidad_coincidentes1)
print("Cantidad de valores coincidentes Coberturas:", cantidad_coincidentes2)
print("Cantidad de valores coincidentes certificados:", cantidad_coincidentes3)
print("Cantidad de valores coincidentes tenencias:", cantidad_coincidentes4)


Cantidad de valores coincidentes Contactos: 3006
Cantidad de valores coincidentes Siniestros: 1595
Cantidad de valores coincidentes Coberturas: 967
Cantidad de valores coincidentes certificados: 77899
Cantidad de valores coincidentes tenencias: 4090


# Estructura de la Secuencia

In [17]:
List = data_cobertura_2022_new.iloc[:,4:].columns
List1 = data_y.iloc[:,:42].columns

In [19]:
def create_sequences(dataframe, sequence_length):
    sequences = []
    unique_clients = dataframe['KEY_CLIENT'].unique()
    
    for client in unique_clients:
        client_data = dataframe[dataframe['KEY_CLIENT'] == client]
        sorted_data = client_data.sort_values(by='PERIODO')
        num_samples = len(sorted_data) - sequence_length + 1
        
        for i in range(num_samples):
            sequence = sorted_data.iloc[i:i+sequence_length]
            sequences.append(sequence)
    
    return sequences

In [20]:
sequence_length = 5
sequences = create_sequences(data_cobertura_2022_new, sequence_length)

In [21]:
X_sequences = []
y_labels = []

for sequence in sequences:
    client_id = sequence['KEY_CLIENT'].values[0]  
    
    client_behavior = data_y[data_y['KEY_CLIENT'] == client_id]
    
    if not client_behavior.empty:
        behavior = client_behavior[['ACCIDENTES_PERSONALES_CORE_1', 'ACCIDENTES_PERSONALES_CORE_2',
       'AERONAVEGACION_CORE_2', 'ART_CORE_2', 'AUI_AUTOS_DE_QBE_CORE_2']].values  
        
        X_sequence = sequence[List].values  
        X_sequences.append(X_sequence)
        y_labels.append(behavior)

X = np.array(X_sequences)
y = np.array(y_labels)


# Datos de Entrenamiento

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.reshape(-1, 5)

# REDES RECURRENTES LSTM

In [31]:
model = Sequential()
model.add(LSTM(64, input_shape=(None, 73)))
model.add(Dense(5, activation='sigmoid'))  
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # Configura el optimizador y la función de pérdida según tu problema


In [32]:
model.fit(X_train, y_train, epochs=10, batch_size=2)

Epoch 1/10
10069/10069 [==============================] - 44s 4ms/step - loss: 0.0366 - accuracy: 0.0540
Epoch 2/10
10069/10069 [==============================] - 31s 3ms/step - loss: 0.0343 - accuracy: 0.0741
Epoch 3/10
10069/10069 [==============================] - 36s 4ms/step - loss: 0.0330 - accuracy: 0.0790
Epoch 4/10
10069/10069 [==============================] - 34s 3ms/step - loss: 0.0318 - accuracy: 0.1048
Epoch 5/10
10069/10069 [==============================] - 37s 4ms/step - loss: 0.0307 - accuracy: 0.1178
Epoch 6/10
10069/10069 [==============================] - 39s 4ms/step - loss: 0.0302 - accuracy: 0.0975
Epoch 7/10
10069/10069 [==============================] - 37s 4ms/step - loss: 0.0287 - accuracy: 0.1088
Epoch 8/10
10069/10069 [==============================] - 32s 3ms/step - loss: 0.0278 - accuracy: 0.1034
Epoch 9/10
10069/10069 [==============================] - 32s 3ms/step - loss: 0.0269 - accuracy: 0.1081
Epoch 10/10
10069/10069 [==============================

In [36]:
model.predict(X_test)

158/158 [==============================] - 1s 3ms/step


array([[5.8594724e-05, 7.8292703e-07, 1.4636646e-15, 9.2552998e-04,
        8.5401858e-08],
       [1.2408349e-02, 1.8848451e-04, 2.1390948e-13, 3.6658989e-03,
        4.0544584e-04],
       [1.3056834e-04, 2.6240519e-05, 1.4109048e-14, 1.3375017e-03,
        6.1171868e-06],
       ...,
       [2.3125917e-04, 5.9001759e-06, 8.0510658e-14, 4.3421201e-03,
        4.2797939e-07],
       [2.2675704e-02, 1.3307073e-04, 1.4795192e-09, 3.9350346e-02,
        1.7943703e-02],
       [1.7815225e-02, 6.5251239e-05, 3.1003921e-08, 2.0847480e-04,
        7.2507246e-05]], dtype=float32)